# Direct Calibration

In [1]:
import numpy as npy
import scipy.optimize as opt
import matplotlib.pyplot as plt

class currentProfile:
    def __init__(self, label):
        self.label=label
        self.current=[]

def dataReader(filename, label):
    inputfile=open(filename,'r')
    outputData=currentProfile(label)
    currentData=[]
    while(1):
        try:
            temp=next(inputfile).split()
            if(len(temp)==5 and temp[4]==str(label)):
                outputData.current.append(1e6*float(temp[2]))
        except StopIteration:
            outputData.current=npy.array(outputData.current)
            break
    return outputData

def fitFunc(time, gamma, omega, phi):
    return gamma*npy.cos(omega*time-phi)**2

labels=["p0","p1","as0","as1"]
measurements=[]
monitors=[]
ratios=[]
fitParameters=[]
fitParametersCOV=[]
minLength=365

dark=dataReader("dataDirectCali/measurement_dark.txt","dark")
for i in range(len(labels)):
    measurements.append(dataReader("dataDirectCali/measurement.txt",labels[i]))
    monitors.append(dataReader("dataDirectCali/monitor.txt",labels[i]))
    ratios.append((measurements[i].current[:minLength]-npy.average(dark.current))/monitors[i].current[:minLength])
    time=npy.array([0.5*j for j in range(minLength)])
    popt,pcov=opt.curve_fit(fitFunc, time, ratios[i], p0=[15,0.01,1])
    fitParameters.append(popt)
    fitParametersCOV.append(pcov)
    fitRatio=fitFunc(time,*popt)
    fig,axs=plt.subplots(2,1)
    axs[0].plot(time,ratios[i],label="Intensity")
    axs[0].plot(time,fitRatio,label="Fitted")
    axs[1].plot(time,(ratios[i]-fitRatio)/ratios[i],label="Residual")
    axs[1].set_xlabel("Time (s)")
    axs[0].set_ylabel("Intensity")
    axs[1].set_ylabel("Residual")
    axs[0].legend()
    axs[1].legend()
    fig.tight_layout()
    fig.savefig("figure/RatioVsTimeDirectCali"+str(i)+"WithDarkdegree.jpg", dpi=1000)
    plt.close()
    
print("Ps="+str(fitParameters[0][2])+"±"+str(npy.sqrt(fitParametersCOV[0][2,2])))
print("As="+str(fitParameters[2][2])+"±"+str(npy.sqrt(fitParametersCOV[2][2,2])))

print("Ps="+str(fitParameters[1][2])+"±"+str(npy.sqrt(fitParametersCOV[1][2,2])))
print("As="+str(fitParameters[3][2])+"±"+str(npy.sqrt(fitParametersCOV[3][2,2])))

Ps=2.2277858560108457±0.0038246903871428945
As=2.354011226435882±0.003199845590851369
Ps=2.225087278090779±0.003585120818196608
As=2.3220409447897117±0.0035016820664099447
